In [4]:
def evaluate_models_in_folder(folder, log_filename, epoch_range, model, valid_loader, device, max_batches):
    """
    Ładuje modele z katalogu 'folder' dla epok określonych w 'epoch_range' i oblicza metryki
    (loss, accuracy, f1_score) na zbiorze walidacyjnym.
    
    Args:
        folder (str): Ścieżka do katalogu z checkpointami.
        log_filename (str): Nazwa pliku logu, do którego zostaną zapisane wyniki.
        epoch_range (iterable): Zakres epok (np. range(1, 11)).
        model (nn.Module): Instancja modelu (musi mieć metodę load_state_dict).
        valid_loader (DataLoader): DataLoader dla zbioru walidacyjnego.
        device (torch.device): Urządzenie, na którym będą wykonywane obliczenia.
        
    Returns:
        dict: Słownik, gdzie klucz to epoka, a wartość to metryki:
              { "loss": avg_loss, "accuracy": accuracy, "f1_score": f1_score }
    """
    import json
    from sklearn.metrics import f1_score
    import torch
    import os
    
    evaluation_results = {}
    loss_function = torch.nn.CrossEntropyLoss()  # założenie – używamy CrossEntropyLoss
    
    for epoch in epoch_range:
        checkpoint_file = os.path.join(folder, f"{model.__class__.__name__}_{epoch}.pth")
        if not os.path.exists(checkpoint_file):
            print(f"Checkpoint file {checkpoint_file} not found, skipping epoch {epoch}")
            continue
        
        checkpoint = torch.load(checkpoint_file, map_location=device)
        model.load_state_dict(checkpoint)
        model.eval()
        
        total_loss = 0.0
        total_samples = 0
        total_correct = 0
        all_labels = []
        all_predictions = []
        batch = 0
        
        with torch.no_grad():
            for images, labels in valid_loader:               
                if batch % 10 == 0:
                    print(f"Batch count {batch}")
                batch += 1
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                if isinstance(outputs, tuple):
                    main_output = outputs[0]
                else:
                    main_output = outputs
                
                loss = loss_function(main_output, labels)
                batch_size = images.size(0)
                total_loss += loss.item() * batch_size
                total_samples += batch_size
                
                max_values, predicted = torch.max(main_output, 1)
                total_correct += (predicted == labels).sum().item()
                all_labels.extend(labels.cpu().tolist())
                all_predictions.extend(predicted.cpu().tolist())
                
                if max_batches and batch == max_batches:
                    break
        
        # Synchronizujemy CUDA, jeśli używamy GPU
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        avg_loss = total_loss / total_samples if total_samples > 0 else 0
        accuracy = total_correct / total_samples if total_samples > 0 else 0
        f1 = f1_score(all_labels, all_predictions, average='macro') if total_samples > 0 else 0
        
        evaluation_results[epoch] = {
            "loss": avg_loss,
            "accuracy": accuracy,
            "f1_score": f1
        }
        print(f"Epoch {epoch}: Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}")
    
    # Zapisujemy wyniki do pliku logu
    log_path = os.path.join(folder, log_filename)
    try:
        with open(log_path, "w") as log_file:
            json.dump(evaluation_results, log_file, indent=4)
        print(f"Saved evaluation log to {log_path}")
    except Exception as error:
        print(f"Could not save evaluation log to {log_path}: {error}")
    
    return evaluation_results


In [1]:
from src.models.our_model1 import OurModelSmall, OurModelBig, OurModel
import torch
from src.utils import load_data, evaluate_model
from src.model_trainer import ModelTrainer
from src.transformations import normalized_simple_transform
from torchsummary import summary
device_str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_str)
torch.manual_seed(123)
print(device)
torch.set_num_threads(12)


cpu


In [2]:
model = OurModel()
valid_loader = load_data('./data/valid', batch_size=200, shuffle=True, transform=normalized_simple_transform(), num_workers=1)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epoch_range = range(1, 11) 
results = evaluate_models_in_folder(
    folder="./saved_models/ourmodel_comp",
    log_filename="evaluation_log.json",
    epoch_range=epoch_range,
    model=model,
    valid_loader=valid_loader,
    device=device,
    max_batches=1
)


Batch count 0
Epoch 1: Loss: 1.1869, Accuracy: 0.5550, F1-score: 0.5357
Checkpoint file ./saved_models/ourmodel_comp\OurModel_2.pth not found, skipping epoch 2


KeyboardInterrupt: 